In [1]:
import pandas as pd
import numpy as np
import warnings

warnings.filterwarnings('ignore')

from sklearn.preprocessing import OneHotEncoder
from sklearn import preprocessing

from sklearn.model_selection import train_test_split
from sklearn.model_selection import TimeSeriesSplit

from sklearn.model_selection import cross_val_score

from optuna import Trial
import optuna
from optuna.samplers import TPESampler

In [2]:
sample_df = pd.read_csv('C:/Projects/python-workspace/Machine Learning/Machine learning project/team4 eda prj/Team4_EDA_ML_Prj/dataset/train_data_ver1.csv', sep=',')

# 시간 순서대로 데이터 반영하기 위해서 거래년월일 정렬 필요
sample_df = sample_df.sort_values('계약날짜인코딩')
sample_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 237987 entries, 64104 to 208092
Data columns (total 13 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   전용면적        237987 non-null  float64
 1   거래금액        237987 non-null  int64  
 2   층           237987 non-null  int64  
 3   건축년도        237987 non-null  int64  
 4   제곱미터단가      237987 non-null  float64
 5   구           237987 non-null  object 
 6   동           237987 non-null  object 
 7   단지명브랜드      237987 non-null  object 
 8   동별 공원 갯수    237987 non-null  float64
 9   8학군         237987 non-null  int64  
 10  동별지하철역수     237987 non-null  int64  
 11  구별 교과학원 갯수  237987 non-null  int64  
 12  계약날짜인코딩     237987 non-null  int64  
dtypes: float64(3), int64(7), object(3)
memory usage: 25.4+ MB


### 1. 무지성 인코딩

In [3]:

'''
    - 카테고리: 데이터 전처리
    - 개요: 무지성 인코딩 => 전체 컬럼 데이터 타입 확인 후 통째로 범주형 인코딩 처리
    - param: df
    - return: encoded columns dataframe
'''
def one_hot(df):
    # 컬럼의 이름 리스트로 뽑아오기
    columns_name_list = list(df.columns)

    # 컬럼마다 for문 반복
    for col_name in columns_name_list:
        # 만약 컬럼의 값 타입이 범주형이면
        if df[col_name].dtype == object or df[col_name].dtype == str:

            # # 컬럼의 유니크한 값을 리스트로 만들어둠
            col_items = df[col_name].unique().tolist()
            # print(col_items)

            onehot = OneHotEncoder(sparse=False)
            # print(col_name)
            onehot_encoded_arr = onehot.fit_transform(df[col_name].values.reshape(-1, 1))
            onehot_encoded_label = onehot.categories_[0]
            onehot_encoded_df = pd.DataFrame(onehot_encoded_arr, columns=onehot_encoded_label)
            # print(onehot_encoded_df)
            df.drop(col_name, axis=1, inplace=True)
            df = pd.concat([df, onehot_encoded_df], axis=1)
        else:
            # 종속(타겟) 컬럼만 인코딩 제외
            if col_name == '제곱미터단가':
                continue
            scaled_label = [col_name]
            x = df[col_name].values.reshape(-1, 1) #returns a numpy array
            min_max_scaler = preprocessing.MinMaxScaler()
            x_scaled = min_max_scaler.fit_transform(x)
            scaled_df = pd.DataFrame(x_scaled, columns=scaled_label)

            df.drop(col_name, axis=1, inplace=True)
            df = pd.concat([df, scaled_df], axis=1)

    return df
'''
    - 카테고리: 데이터 전처리
    - 개요: 무지성 인코딩 => 새로운 데이터 컬럼 추가 시 별도 인코딩 결과만 출력할 때 사용 ㄱㄱ
    - param: df, col_name
    - return: encoded columns dataframe or series
'''
def encode_column(df, col_name):

    # column data type이 object 또는 str 즉 범주형일 경우
    # onehot 인코딩 수행된 데이터프레임 return
    if df[col_name].dtype == object or df[col_name].dtype == str:
        onehot = OneHotEncoder(sparse=False)

        onehot_encoded_arr = onehot.fit_transform(df[col_name].values.reshape(-1, 1))
        onehot_encoded_label = onehot.categories_[0]
        onehot_encoded_df = pd.DataFrame(onehot_encoded_arr, columns=onehot_encoded_label)

        return onehot_encoded_df

    # column data type이 나머지 타입일 경우
    # 해당 컬럼의 series return
    else:
        return df[col_name]

In [4]:
# 1-1 test one_hot function
preprocessed_train_df = one_hot(sample_df)
# print(len(preprocessed_train_df.columns))
preprocessed_train_df.head()

,제곱미터단가,전용면적,거래금액,층,건축년도,강남구,강동구,강북구,강서구,관악구,...,한라|한라비발디,한화,현대,호반,힐스테이트,동별 공원 갯수,8학군,동별지하철역수,구별 교과학원 갯수,계약날짜인코딩
0,1625.609604,0.173928,0.149293,0.044118,0.433333,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.533333,1.0,0.142857,1.0,0.001826
1,1463.048643,0.272890,0.133717,0.014706,0.433333,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.533333,1.0,0.142857,1.0,0.006393
2,1625.609604,0.272032,0.149293,0.000000,0.433333,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.533333,1.0,0.142857,1.0,0.009132
3,1744.404152,0.272524,0.160676,0.014706,0.433333,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.533333,1.0,0.142857,1.0,0.070320
4,1955.256457,0.270315,0.122334,0.058824,0.433333,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.533333,1.0,0.142857,1.0,0.085845


### 2. 모델링

In [5]:
'''
    - 카테고리: 모델링
    - 개요: 머신러닝 모델링 수행 및 점수 도출
        - 교차 검증 방법으로 TimeSeriesSplit 수행
    - param: 

        1. model_tuple => ex. ('LR', LinearRegression())
        2. X_train, y_train, X_test, y_test

    - return: rmse
'''
def execute_modeling(model_tuple, data, target):

    name = model_tuple[0]
    model = model_tuple[1]

    # 각 모델에 대하여 실질적 학습 수행
    # clf = model.fit(X_train, y_train)
    # pred = clf.predict(X_test)

    # TimeSeries Cross validation 
    tscv = TimeSeriesSplit(n_splits=15)

    # 각 모델에 대하여 교차 검증한 결과 점수 확인
    # scoring parameter option 어캐 줘야 함?
    rmse_scores = cross_val_score(model, data, target, cv=tscv, scoring='neg_mean_squared_error')
    rmse_scores = np.sqrt(-rmse_scores)
    print(f'{name} average rmse_scores: {rmse_scores.mean()}, rmse_scores: {rmse_scores}')

    return rmse_scores.mean()

In [7]:
from sklearn.linear_model import LinearRegression 
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb
import lightgbm as lgb

# 2. test execute_modeling function
preprocessed_train_df = one_hot(sample_df)
preprocessed_train_df = preprocessed_train_df.drop_duplicates()

# 시간 순서대로 데이터 반영하기 위해서 거래년월일 정렬 필요
# preprocessed_train_df = preprocessed_train_df.sort_values('거래년월일')

# 인코딩된 데이터에서 data, target 분류
data = preprocessed_train_df[preprocessed_train_df.columns.difference(['제곱미터단가', '거래금액', '전용면적'])]
target = preprocessed_train_df['제곱미터단가'] 

model_list = [
                # ('LR', LinearRegression()), 
                ('RF', RandomForestRegressor(n_estimators = 10))
                # ('model_xgb', xgb.XGBRegressor(n_estimators=500, max_depth=9, min_child_weight=5, gamma=0.1, n_jobs=-1)),
                # ('model_lgb', lgb.LGBMRegressor(n_estimators=500, max_depth=9, min_child_weight=5, n_jobs=-1))
            ]

for model_tuple in model_list:
    execute_modeling(model_tuple, data, target)
    pass

RF average rmse_scores: 313.09799883402644, rmse_scores: [213.39864626 258.85504269 581.40359519 574.01418487 314.94635758
 154.41631554 196.5630953  264.79658547 493.89245269 298.34507594
 232.93599578 194.03355357 292.31163995 342.37758341 284.17985827]


### 3. 파라미터 튜닝

In [7]:
# 3. test get_best_param function

# linear regression model object
def linear_object(trial:Trial, data, target):
    params = {
        'fit_intercept': trial.suggest_categorical('fit_intercept', [True, False])
    }
    
    test_model = LinearRegression(**params)
    test_model_score = execute_modeling(('LR', test_model), data, target)

    return test_model_score

# xgbr regression model object
def xgbr_object(trial:Trial, data, target):
    params = {
        "n_estimators" : trial.suggest_int('n_estimators', 500, 1000),
        'max_depth':trial.suggest_int('max_depth', 8, 16),
        'min_child_weight':trial.suggest_int('min_child_weight', 1, 100),
        'gamma':trial.suggest_int('gamma', 1, 3),
        'learning_rate': 0.01,
        'colsample_bytree':trial.suggest_discrete_uniform('colsample_bytree',0.5, 1, 0.1),
        'nthread' : -1,
        'tree_method': 'gpu_hist',
        'predictor': 'gpu_predictor',
        'lambda': trial.suggest_loguniform('lambda', 1e-3, 1.0),
        'alpha': trial.suggest_loguniform('alpha', 1e-3, 1.0),
        'subsample': trial.suggest_categorical('subsample', [0.6,0.7,0.8,1.0] ),
        'random_state': 42
    }
    
    test_model = xgb.XGBRegressor(**params)
    test_model_score = execute_modeling(('XGBR', test_model), data, target)

    return test_model_score

# light gbm regression model object
def lgb_object(trial:Trial, data, target):
    params = {
        "n_estimators" : trial.suggest_int('n_estimators', 500, 1000),
        'max_depth':trial.suggest_int('max_depth', 8, 16),
        'min_child_weight':trial.suggest_int('min_child_weight', 1, 100)
    }
    
    test_model = lgb.LGBMRegressor(**params)
    test_model_score = execute_modeling(('LGBM', test_model), data, target)

    return test_model_score

# 하이퍼 파라미터 결과 도출
# 위에서 분할한  X_train, y_train, X_test, y_test 파라미터 삽입
study = optuna.create_study(direction='minimize', sampler=TPESampler())
study.optimize(lambda trial: lgb_object(trial, data, target), n_trials=3)

best_score = study.best_value
best_param_dict = study.best_trial.params

print(best_score, best_param_dict)

[I 2021-12-28 16:21:42,984] A new study created in memory with name: no-name-aa9728d7-5b52-4b7f-a314-9ab0fc9daa4f
[I 2021-12-28 16:22:21,732] Trial 0 finished with value: 25895.329099756465 and parameters: {'n_estimators': 790, 'max_depth': 14, 'min_child_weight': 43}. Best is trial 0 with value: 25895.329099756465.


LGBM average rmse_scores: 25895.329099756465, rmse_scores: [19576.06321571 19376.58158036 55598.44322939 56276.78036999
 29394.86794348 12642.15597452 12019.52356509 25045.80398683
 21638.95337171 34075.15963913 14420.76480632 13809.82873753
 21766.47121174 26651.99570745 26136.5431571 ]


[I 2021-12-28 16:23:02,077] Trial 1 finished with value: 26214.330928425254 and parameters: {'n_estimators': 750, 'max_depth': 9, 'min_child_weight': 64}. Best is trial 0 with value: 25895.329099756465.


LGBM average rmse_scores: 26214.330928425254, rmse_scores: [19831.40428064 20039.21382514 54863.88110247 56009.72590423
 29900.24350841 13507.73394071 12359.79148079 25557.985213
 22598.22387112 34117.80646005 14865.52649845 13909.8459276
 21980.87217221 27063.63479704 26609.07494452]


[I 2021-12-28 16:23:40,147] Trial 2 finished with value: 25739.27317819786 and parameters: {'n_estimators': 727, 'max_depth': 14, 'min_child_weight': 21}. Best is trial 2 with value: 25739.27317819786.


LGBM average rmse_scores: 25739.27317819786, rmse_scores: [19982.4711522  19401.4453833  55313.02189458 56188.8881972
 29589.53745901 12440.62745936 11658.56881514 24586.18736491
 21635.68308552 33538.90012906 14624.16575483 13800.14089515
 21483.4692944  26478.39201484 25367.59877348]
25739.27317819786 {'n_estimators': 727, 'max_depth': 14, 'min_child_weight': 21}


In [8]:
# 하이퍼파라미터별 중요도를 확인할 수 있는 그래프
optuna.visualization.plot_param_importances(study)

In [9]:
# 하이퍼파라미터 최적화 과정을 확인
optuna.visualization.plot_optimization_history(study)